In [13]:
import numpy as np
from scipy.sparse import csr_matrix
import pdb
from time import time
import cPickle as pickle
import os
# --------
import theano
from theano import tensor as T
import lasagne
from lasagne.layers import *
from lasagne.objectives import *
from lasagne.nonlinearities import *
from lasagne.updates import *
from lasagne.utils import *
from lasagne.regularization import *
from lasagne.init import *

import itertools

In [3]:
from lasagne import init
from lasagne import nonlinearities

class DenseLowRankLayer(Layer):
    def __init__(self, incoming, num_units, W1=init.GlorotUniform(), W2=init.GlorotUniform(), k=1,
                 b=init.Constant(0.), nonlinearity=nonlinearities.rectify,
                 num_leading_axes=1, **kwargs):
        super(DenseLowRankLayer, self).__init__(incoming, **kwargs)
        self.nonlinearity = (nonlinearities.identity if nonlinearity is None
                             else nonlinearity)

        self.num_units = num_units
        self.k = k
        
        if num_leading_axes >= len(self.input_shape):
            raise ValueError(
                    "Got num_leading_axes=%d for a %d-dimensional input, "
                    "leaving no trailing axes for the dot product." %
                    (num_leading_axes, len(self.input_shape)))
        elif num_leading_axes < -len(self.input_shape):
            raise ValueError(
                    "Got num_leading_axes=%d for a %d-dimensional input, "
                    "requesting more trailing axes than there are input "
                    "dimensions." % (num_leading_axes, len(self.input_shape)))
        self.num_leading_axes = num_leading_axes

        if any(s is None for s in self.input_shape[num_leading_axes:]):
            raise ValueError(
                    "A DenseLayer requires a fixed input shape (except for "
                    "the leading axes). Got %r for num_leading_axes=%d." %
                    (self.input_shape, self.num_leading_axes))
        num_inputs = int(np.prod(self.input_shape[num_leading_axes:]))
        
        # W_(num inputs x num_units) = W1_(num_inputs x m) * W2_(m x num_units)

        self.W1 = self.add_param(W1, (num_inputs, k), name="W1")
        self.W2 = self.add_param(W2, (k, num_units), name="W2")
        
        if b is None:
            self.b = None
        else:
            self.b = self.add_param(b, (num_units,), name="b",
                                    regularizable=False)

    def get_output_shape_for(self, input_shape):
        return input_shape[:self.num_leading_axes] + (self.num_units,)

    def get_output_for(self, input, **kwargs):
        num_leading_axes = self.num_leading_axes
        if num_leading_axes < 0:
            num_leading_axes += input.ndim
        if input.ndim > num_leading_axes + 1:
            # flatten trailing axes (into (n+1)-tensor for num_leading_axes=n)
            input = input.flatten(num_leading_axes + 1)

        activation = T.dot(input, T.dot(self.W1, self.W2))
        if self.b is not None:
            activation = activation + self.b
        return self.nonlinearity(activation)

In [4]:
l_in = InputLayer((None, 100))
l_dense = DenseLowRankLayer(l_in, k=5, num_units=50)
l_inv = DenseLowRankLayer(l_dense, k=5, num_units=100)

In [5]:
l_inv.output_shape

(None, 100)

In [6]:
l_inv.W1.get_value().shape

(50, 5)

In [7]:
l_inv.W2.get_value().shape

(5, 100)

In [14]:
l_in2 = InputLayer((None, 200))
l_dense2 = DenseLowRankLayer(l_in2, k=5, num_units=50, W2=l_dense.W2)
l_inv2 = DenseLowRankLayer(l_dense2, k=5, num_units=200, W1=l_inv.W1)

AsTensorError: ('Cannot convert <lasagne.init.GlorotUniform object at 0x10d200f50> to TensorType', <class 'lasagne.init.GlorotUniform'>)

In [9]:
l_inv2.W1.get_value().shape

(50, 5)

In [10]:
l_inv2.W2.get_value().shape

(5, 200)

In [11]:
l_dense.W2 == l_dense2.W2

True